In [12]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import cv2 
import os
import fnmatch
from sklearn import svm
from sklearn.metrics import accuracy_score
from keras.models import Sequential, Model
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Conv2D, MaxPooling2D
from keras import layers
from tensorflow import keras
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [13]:
train_path = "Demo_dataset"
train_chinese = os.listdir(train_path)
df_train = pd.DataFrame(columns=["character", "filename", "label"])
df_test = pd.DataFrame(columns=["character", "filename", "label"])
len(train_chinese)

8

In [14]:
def image_preprocessing(df_train):
    array = np.zeros((df_train.shape[0], 128, 128, 3))
    for i in range(df_train.shape[0]):
        if df_train.iloc[i]['filename'] != ".DS_Store":
            image_path = train_path + '/' + df_train.iloc[i]['character'] + '/' + df_train.iloc[i]['filename']
            # get grayscale image
            # print(image_path)
            image = cv2.imread(image_path)
            # remove noise
            out = cv2.medianBlur(image, 5)
            # normalize the image
            resultimage = np.zeros(out.shape)
            normalizedimage = cv2.normalize(out, resultimage, 0, 255, cv2.NORM_MINMAX)
            # resize the image 
            resizeimage = cv2.resize(normalizedimage, (128,128))

            blue, green, red = cv2.split(resizeimage) 
            # apply principal component analysis
            pca = PCA(50)
            red_transformed = pca.fit_transform(red)
            red_inverted = pca.inverse_transform(red_transformed)

            #Applying to Green channel and then applying inverse transform to transformed array.
            green_transformed = pca.fit_transform(green)
            green_inverted = pca.inverse_transform(green_transformed)

            #Applying to Blue channel and then applying inverse transform to transformed array.
            blue_transformed = pca.fit_transform(blue)
            blue_inverted = pca.inverse_transform(blue_transformed)

            img_compressed = (np.dstack((red_inverted, red_inverted, red_inverted))).astype(np.uint8)
            array[i] = img_compressed
    return array

In [15]:
k = 0
for i in range(0, len(train_chinese)):
    if i != ".DS_Store":
        l = os.listdir(train_path + '/' + train_chinese[i])
        l = fnmatch.filter(l, '*.png')
        l = np.array(l)
        kf = KFold(n_splits = 5, random_state = None, shuffle = False)
        for j, (train_index, test_index) in enumerate(kf.split(l)):
            if len(l) >= 200:
                l = l[:200]
                label_train = [k] * len(l[train_index])
                label_test = [k] * len(l[test_index])
                temp1 = pd.DataFrame({
                    "character": train_chinese[i],
                    "filename": l[train_index],
                    "label": label_train
                })
                temp2 = pd.DataFrame({
                    "character": train_chinese[i],
                    "filename": l[test_index],
                    "label": label_test
                })
                #print(temp)
                df_train = pd.concat([df_train, temp1])  
                df_test = pd.concat([df_test, temp2])
        k += 1
                

In [16]:
train_array = image_preprocessing(df_train)
test_array = image_preprocessing(df_test)
#print(train_array.shape)
#print(test_array.shape)

X_train = train_array / 255.0
y_train = df_train["label"].to_numpy()
y_train = y_train.astype('int')


X_test = test_array / 255.0
y_test = df_test["label"].to_numpy()
y_test = y_test.astype('int')

In [17]:
train_X = np.zeros((5, X_train.shape[0] // 800 * 160, 128, 128, 3))
train_y = np.zeros((5, y_train.shape[0] // 800 * 160))
test_X = np.zeros((5, X_test.shape[0] // 200 * 40, 128, 128, 3))
test_y = np.zeros((5, y_test.shape[0] // 200 * 40))


In [18]:
for i in range(8):
    x1 = X_train[800 * i : 800 * (i + 1)]
    y1 = y_train[800 * i : 800 * (i + 1)]
    x2 = X_test[200 * i : 200  * (i + 1)]
    y2 = y_test[200 * i : 200 * (i + 1)]
    for j in range(5):
        #print(i, j)
        #print(160 * i, 160 * (i+1))
        train_X[j][160 * i  : 160 * (i+1)]  = x1[160 * j : 160 * (j + 1)]
        train_y[j][160 * i  : 160 * (i+1)]  = y1[160 * j : 160 * (j + 1)]
        test_X[j][40 * i : 40 * (i + 1)] = x2[40 * j : 40  * (j + 1)]
        test_y[j][40 * i : 40 * (i + 1)] = y2[40 * j : 40  * (j + 1)]

In [19]:
models = []
for i in range(5):
    model = keras.models.Sequential([

        keras.layers.Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=(3,3)),
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.Flatten(),

        keras.layers.Dense(1024,activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1024,activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(8,activation='softmax')  

    ])
    models.append(model)
    
        
    

In [20]:
accuracy = []
for i in range(5):
    models[i].compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.optimizers.SGD(learning_rate=0.001),
        metrics=['accuracy']    
    )
    history=models[i].fit(
        train_X[i], train_y[i],
        epochs = 15,
        validation_data = (test_X[i], test_y[i]),
        validation_freq = 1
    )
    accuracy.append(np.max(history.history['val_accuracy']))
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/15
40/40 [==============================] - 6s 158ms/step - loss: 2.5726 - accuracy: 0.1672 - val_loss: 2.0826 - val_accuracy: 0.1219
Epoch 2/15
40/40 [==============================] - 6s 149ms/step - loss: 2.1463 - accuracy: 0.2508 - val_loss: 2.0879 - val_accuracy: 0.1250
Epoch 3/15
40/40 [==============================] - 6s 154ms/step - loss: 1.9529 - accuracy: 0.3195 - val_loss: 2.0906 - val_accuracy: 0.1375
Epoch 4/15
40/40 [==============================] - 6s 158ms/step - loss: 1.7441 - accuracy: 0.3430 - val_loss: 2.0796 - val_accuracy: 0.1437
Epoch 5/15
40/40 [==============================] - 7s 175ms/step - loss: 1.6192 - accuracy: 0.4078 - val_loss: 2.0434 - val_accuracy: 0.2562
Epoch 6/15
40/40 [==============================] - 7s 174ms/step - loss: 1.4710 - accuracy: 0.4656 - val_loss: 1.9482 - val_accuracy: 0.3187
Epoch 7/15
40/40 [==============================] - 7s 183ms/step - loss: 1.3721 - accuracy: 0.5180 - val_loss: 1.7744 - val_accuracy: 0.3250
Epoch 

40/40 [==============================] - 7s 179ms/step - loss: 0.8648 - accuracy: 0.7070 - val_loss: 0.7712 - val_accuracy: 0.7906
Epoch 14/15
40/40 [==============================] - 7s 174ms/step - loss: 0.7717 - accuracy: 0.7602 - val_loss: 0.5990 - val_accuracy: 0.8375
Epoch 15/15
40/40 [==============================] - 7s 175ms/step - loss: 0.7467 - accuracy: 0.7414 - val_loss: 0.5781 - val_accuracy: 0.8500
Epoch 1/15
40/40 [==============================] - 7s 179ms/step - loss: 2.7173 - accuracy: 0.1328 - val_loss: 2.0807 - val_accuracy: 0.1250
Epoch 2/15
40/40 [==============================] - 7s 169ms/step - loss: 2.2079 - accuracy: 0.2359 - val_loss: 2.0776 - val_accuracy: 0.1000
Epoch 3/15
40/40 [==============================] - 7s 169ms/step - loss: 1.9960 - accuracy: 0.2891 - val_loss: 2.0624 - val_accuracy: 0.1500
Epoch 4/15
40/40 [==============================] - 7s 172ms/step - loss: 1.7954 - accuracy: 0.3539 - val_loss: 2.0508 - val_accuracy: 0.2000
Epoch 5/15
40/4

In [21]:
mean_accuracy = np.mean(accuracy)
mean_accuracy

0.840625011920929